```
@作者: 阿凯爱玩机器人
@QQ: 244561792
@微信: xingshunkai
@邮箱: xingshunkai@qq.com
@网址: deepsenserobot.com
@B站: "阿凯爱玩机器人"
```

## 导入依赖

In [1]:
import copy
import numpy as np
import cv2
import open3d as o3d
# 自定义库
from kyle_robot_toolbox.open3d import *

np.set_printoptions(precision=3, suppress=True)

pybullet build time: Jun  3 2022 02:05:55


## 载入点云

In [2]:
# 载入盒子表面点云
# 去除离群点后的
box_panel_pcd = o3d.io.read_point_cloud("./data/example/box_pose/box_panel_filter.pcd")
# 赋值为灰色
box_panel_pcd.paint_uniform_color([0.5, 0.5, 0.5])
# 点云可视化
draw_geometry([box_panel_pcd], window_name="盒子上表面点云")

## 计算点云AABB包围盒

In [3]:
# 点云AABB包围盒
aabb = box_panel_pcd.get_axis_aligned_bounding_box()
aabb.color = [1, 0, 0] # 红色

# 绘制AABB
o3d.visualization.draw_geometries([box_panel_pcd, aabb], window_name="点云AABB包围盒",
                                  width=800,  # 窗口宽度
                                  height=600)  # 窗口高度

In [4]:
# 包围盒,最小值坐标 [x_min, y_min, z_min]
print(f"min_bound: {aabb.min_bound}")
x_min, y_min, z_min = aabb.min_bound
print(f"x_min={x_min:.4f} y_min={y_min:.4f} z_min={z_min:.4f}, 单位m")

min_bound: [ 0.033 -0.013  0.222]
x_min=0.0328 y_min=-0.0129 z_min=0.2220, 单位m


In [5]:
print(f"max_bound: {aabb.max_bound}")
x_max, y_max, z_max = aabb.max_bound
print(f"x_max={x_max:.4f} y_max={y_max:.4f} z_max={z_max:.4f}, 单位m")

max_bound: [0.064 0.017 0.226]
x_max=0.0642 y_max=0.0166 z_max=0.2260, 单位m


In [6]:
# AABB包围盒的尺寸
length = x_max - x_min
width = y_max - y_min 
height = z_max - z_min
print(f"AABB包围盒尺寸: 长: {length:.4f} 宽: {width:.4f} 高: {height:.4f}")

AABB包围盒尺寸: 长: 0.0314 宽: 0.0295 高: 0.0040


## 计算OBB包围盒
OBB为旋转包围盒

In [7]:
# 点云OBB包围盒
robust = True # 鲁棒开关， 实际测试robust=True效果反而会不好
obb = box_panel_pcd.get_oriented_bounding_box(robust=robust)
obb.color = [0, 1, 0] # 绿色

# 绘制OBB
o3d.visualization.draw_geometries([box_panel_pcd, obb], window_name="点云OBB包围盒",
                                  width=800,  # 窗口宽度
                                  height=600)  # 窗口高度

In [8]:
print(f"OBB包围盒的维度: {obb.dimension()}")

OBB包围盒的维度: 3


In [9]:
print(f"OBB包围盒中心点: {obb.center}")

OBB包围盒中心点: [0.048 0.001 0.225]


In [10]:
length, width, height = obb.extent
print(f"OBB包围盒的尺寸: 长: {length:.4f} 宽: {width:.4f} 高: {height:.4f}")

OBB包围盒的尺寸: 长: 0.0371 宽: 0.0372 高: 0.0021


In [11]:
# 包围盒的8个角点
print(np.asarray(obb.get_box_points()))

[[ 0.022  0.002  0.226]
 [ 0.047 -0.025  0.224]
 [ 0.049  0.028  0.223]
 [ 0.022  0.002  0.228]
 [ 0.074  0.001  0.223]
 [ 0.049  0.028  0.225]
 [ 0.047 -0.025  0.226]
 [ 0.074  0.001  0.221]]


## AABB与OBB二者对比

In [12]:
o3d.visualization.draw_geometries([box_panel_pcd, aabb, obb], window_name="点云AABB与OBB包围盒",
                                  width=800,  # 窗口宽度
                                  height=600)  # 窗口高度